# Running MAGEMin calculations through Julia

PetThermoTools provides the ability to run calculations/models that utilize the recent Weller et al. (2024) and Green et al. (2025) thermodynamic models. Once everything is set up you should be able to run these calculations from the same functions used to run models with alphaMELTS, but simply setting the ```Model``` kwarg to ```Green2025``` or ```Weller2024```.

I've worked hard to try and ensure a set up routine that works on every operating system, regardless of prior installations etc. There is, however, a good chance you'll come across a bug/issue I haven't seen before. If there is an issue in the following script please email me (gleesonm@berkeley.edu) and I will try to find a fix as quickly as possible!

**Please read the markdown cells carefully for instructions before each code cell**. If everything goes well you should be able to run calculations using the Weller et al. (2024) and Green et al. (2025) thermodynamic models!

### Step 1 
Uncomment the line below and install juliacall in your Python environment

In [1]:
# !pip install juliacall

### Step 2
Import Main as jl from juliacall. This means that we can call julia functions directly from Python by typing ```jl.{function_name}```.

In [1]:
from juliacall import Main as jl, convert as jlconvert

### Step 3
To ensure that this is going to work we need to remove any pre-existing julia packages that might interfere with ```PythonCall```. The main culprits are ```PyCall``` and ```Pandas```. If you are new to Julia or haven't done extensive work in Julia before, the easiest way to do this is to set `option1 = True` in the cell below. 

**If you use Julia and don't want to remove any other packages that you might need then please put `option2 = True` and `option1 = False`. This will just remove PyCall and Pandas.** If you select option 2 and then get issues later on in this notebook please let me know. If Pandas and/or PyCall are not installed in your Julia environment option 2 might return an error. This isn't as issue as long as neither of these packages are installed. If you're unsure what option to select and don't use Julia for other purposes, the safest option is to select option 1.

In [2]:
option1 = True
option2 = False
if option1:
    jl.seval("""
    using Pkg

    for pkg in values(Pkg.dependencies())  # Get metadata values instead of UUIDs
        if pkg.is_direct_dep  # Only remove explicitly installed packages
            Pkg.rm(pkg.name)  # Use the package name instead of UUID
        end
    end

    Pkg.gc()  # Clean up unused dependencies

    """)
elif option2:
    jl.seval("Using Pkg")
    jl.Pkg.rm("PyCall")
    jl.Pkg.rm("Pandas")

### Step 4
Install ```PythonCall``` in Julia.

In [3]:
jl.Pkg.add("PythonCall")

### Step 5
Next we need to install MAGEMinCalc (my code) which will also install the underlying MAGEMin_C code (Riel et al. 2022). I've encountered some issues on Windows with this step, so if you originally get an error similar to that listed below then uncomment the line ```jl.seval('ENV["JULIA_SSL_CA_ROOTS_PATH"] = ""')``` and rerun this cell.

```JuliaError: failed to clone from https://github.com/gleesonm1/MAGEMinCalc.git, error: GitError(Code:ERROR, Class:SSL, Your Julia is built with a SSL/TLS engine that libgit2 doesn't know how to configure to use a file or directory of certificate authority roots, but your environment specifies one via the SSL_CERT_FILE variable. If you believe your system's root certificates are safe to use, you can export JULIA_SSL_CA_ROOTS_PATH="" in your environment to use those instead.)```

In [4]:
# Set the environment variable inside Julia - uncomment if the code below returns an error.
# jl.seval('ENV["JULIA_SSL_CA_ROOTS_PATH"] = ""')

# Install MAGEMinCalc
jl.Pkg.add(url="https://github.com/gleesonm1/MAGEMinCalc.git", rev="v0.4.1")

### Step 6
Precompile the Julia packages to check for any issues.

In [5]:
jl.Pkg.precompile()

### Step 7
Test the installation by running the following cell. This is calling the MAGEMinCalc functions directly with Julia code using juliacall. This is different to how the code is usually initiated in PetThermoTools but it provides an initial test of the installation. If successful it should print a Pandas DataFrame containing the predicted melt compositions in this crystallization example:

In [6]:
jl.seval("""
using MAGEMinCalc

comp = Dict("SiO2_Liq" => 47.5, "Al2O3_Liq" => 16.4, "CaO_Liq" => 11.6, "MgO_Liq" => 9.38,
            "FeOt_Liq" => 9.16, "K2O_Liq" => 0.329, "Na2O_Liq" => 2.25, "TiO2_Liq" => 2.29, 
            "Fe3Fet_Liq" => 0.15, "Cr2O3_Liq" => 0.0, "H2O_Liq" => 0.68)

Results = MAGEMinCalc.path(comp = comp, T_end_C = 1100.0, dt_C = 2.0, 
            P_bar = 1000.0, frac_xtal = true, 
            Model = "ig",
            find_liquidus = true)

""")

jl.Results["liq1"]

## Step 8 - You need to restart your kernal before running any of the code below!
Assuming the crystallization model above ran without any issues we can now progress to testing the integration of MAGEMin with PetThermoTools. First make sure you have the up to date version of PetThermoTools installed (v0.2.32 or later).

In [7]:
# !pip install --upgrade PetThermoTools

In [8]:
import PetThermoTools as ptt
ptt.__version__

### Step 9
We are now going to run the same crystallization model as we did above, but this time through PetThermoTools. You don't need to change anything, the two cells of code below should run the model and plot a simple graph of MgO vs FeOt. Expect the code to take 10 - 60 seconds depending on your computer performance.

In [9]:
comp = {"SiO2_Liq": 47.5, 
        "Al2O3_Liq": 16.4, 
        "CaO_Liq": 11.6, 
        "MgO_Liq": 9.38,
        "FeOt_Liq": 9.16, 
        "K2O_Liq": 0.329, 
        "Na2O_Liq": 2.25, 
        "TiO2_Liq": 2.29,
        "Fe3Fet_Liq": 0.15, 
        "Cr2O3_Liq": 0.0, 
        "H2O_Liq": 0.68}

Results = ptt.isobaric_crystallisation(bulk=comp,
                             Model = "Green",
                             dt_C = 2.0,
                             T_end_C=1100.0,
                             P_bar=1000.0,
                             find_liquidus=True,
                             Frac_solid=True)

In [10]:
import matplotlib.pyplot as plt
plt.plot(Results["All"]["MgO_Liq"], Results["All"]["FeOt_Liq"], '-r')
plt.xlabel('MgO')
plt.ylabel('FeO$_t$')

### Step 10 - use PetThermoTools alongside MAGEMin in your research!